In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import random as rand
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import train_test_split
from __future__ import division
from scipy.sparse import hstack

C:\Users\Hicham\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#Import Predictions
LR = pd.read_csv('Predictions/logisticRegression.csv')
KNN = pd.read_csv('Predictions/KNN.csv')
DT = pd.read_csv('Predictions/DecisionTree.csv')
RF = pd.read_csv('Predictions/RandomForest.csv')
GB = pd.read_csv('Predictions/GradientBoosting.csv')
AdB = pd.read_csv('Predictions/AdaBoost.csv')  
NB = pd.read_csv('Predictions/NaiveBayes.csv')
NN = pd.read_csv('Predictions/NeuralNetwork.csv')

In [3]:
Test = pd.read_csv('Test.csv')

In [4]:
def dropDups(df):
    L =[]
    for i in xrange(0,df.columns.values.size-1) : 
        for y in xrange(i+1,df.columns.values.size-1) :
            if(np.array_equal( np.array(df[df.columns[i]]), np.array(df[df.columns[y]]))):
                L.append(df.columns[y])
    return L

In [5]:
LR  = LR.drop(dropDups(LR), axis=1)
KNN = KNN.drop(dropDups(KNN), axis=1)
RF  = RF.drop(dropDups(RF), axis=1)
GB  = GB.drop(dropDups(GB), axis=1)
AdB = AdB.drop(dropDups(AdB), axis=1)
NB  = NB.drop(dropDups(NB), axis=1)
NN  = NN.drop(dropDups(NN), axis=1)
DT = DT.drop(dropDups(NN), axis=1)

In [6]:
DataFrames = [RF, GB, AdB, DT, NN, KNN, LR, NB]

In [7]:
dataFrame = pd.concat(DataFrames, axis=1)
dataFrame.shape

(15204, 299)

In [8]:
dataFrame = dataFrame.rename(columns = {'Unnamed: 0':'ID'})
dataFrame = dataFrame.drop('ID', axis=1)
dataFrame.head()

,"RandomForest_1_[('n_estimators', 1300), ('criterion', 'gini'), ('max_depth', 13), ('min_samples_leaf', 1)]","RandomForest_2_[('n_estimators', 250), ('criterion', 'gini'), ('max_depth', 10), ('min_samples_leaf', 9)]","RandomForest_3_[('n_estimators', 1150), ('criterion', 'entropy'), ('max_depth', 15), ('min_samples_leaf', 11)]","RandomForest_4_[('n_estimators', 100), ('criterion', 'entropy'), ('max_depth', 13), ('min_samples_leaf', 1)]","RandomForest_5_[('n_estimators', 150), ('criterion', 'gini'), ('max_depth', 5), ('min_samples_leaf', 9)]","RandomForest_6_[('n_estimators', 650), ('criterion', 'entropy'), ('max_depth', 5), ('min_samples_leaf', 11)]","RandomForest_7_[('n_estimators', 750), ('criterion', 'gini'), ('max_depth', 2), ('min_samples_leaf', 1)]","RandomForest_8_[('n_estimators', 800), ('criterion', 'gini'), ('max_depth', 2), ('min_samples_leaf', 3)]","RandomForest_9_[('n_estimators', 1250), ('criterion', 'gini'), ('max_depth', 1), ('min_samples_leaf', 5)]","RandomForest_10_[('n_estimators', 250), ('criterion', 'entropy'), ('max_depth', 1), ('min_samples_leaf', 9)]",...,"logisticRegression_99_[('C', 8.8000000000000007)]","logisticRegression_100_[('C', 20.0)]","logisticRegression_102_[('C', 3.2000000000000002)]","logisticRegression_107_[('C', 8.4000000000000004)]","logisticRegression_110_[('C', 17.400000000000002)]","logisticRegression_115_[('C', 1.8)]","logisticRegression_118_[('C', 18.0)]","logisticRegression_119_[('C', 24.400000000000002)]","logisticRegression_121_[('C', 18.199999999999999)]",NaiveBayes
0,0.114129,0.105622,0.101529,0.110790,0.100833,0.104548,0.091496,0.093300,0.074148,0.072151,...,0.073316,0.073309,0.073338,0.073316,0.073309,0.073365,0.073309,0.073308,0.073309,0.991966
1,0.007052,0.007641,0.007125,0.007090,0.009816,0.009173,0.015646,0.015422,0.021654,0.020207,...,0.034153,0.034144,0.034181,0.034154,0.034145,0.034215,0.034145,0.034143,0.034145,0.983712
2,0.022584,0.021445,0.021615,0.025307,0.021100,0.026020,0.024748,0.025035,0.027486,0.030436,...,0.029076,0.029070,0.029093,0.029076,0.029071,0.029113,0.029071,0.029069,0.029071,0.983930
3,0.012240,0.011751,0.006462,0.006775,0.017668,0.016402,0.024315,0.024665,0.027486,0.030436,...,0.012523,0.012518,0.012539,0.012524,0.012519,0.012559,0.012519,0.012518,0.012519,0.926150
4,0.205486,0.199152,0.226337,0.206944,0.144274,0.145915,0.096615,0.097248,0.074148,0.072404,...,0.055647,0.055644,0.055655,0.055647,0.055644,0.055665,0.055644,0.055644,0.055644,0.988173


In [9]:
Correlation = dataFrame.corr()

In [10]:
couplesHighCorr = np.where(Correlation >= 0.7)
couplesHighCorr = [[Correlation.index[x], Correlation.columns[y]] for x, y in zip(*couplesHighCorr) if x != y and x < y]
len(couplesHighCorr)

15408

In [11]:
lowcorr = np.where(Correlation < 0.7)
lowcorr = [[Correlation.index[a], Correlation.columns[b]] for a, b in zip(*lowcorr) if a != b and a < b]
len(lowcorr)

26497

In [17]:
#Models names
header = list(dataFrame)
header

["RandomForest_1_[('n_estimators', 1300), ('criterion', 'gini'), ('max_depth', 13), ('min_samples_leaf', 1)]",
 "RandomForest_2_[('n_estimators', 250), ('criterion', 'gini'), ('max_depth', 10), ('min_samples_leaf', 9)]",
 "RandomForest_3_[('n_estimators', 1150), ('criterion', 'entropy'), ('max_depth', 15), ('min_samples_leaf', 11)]",
 "RandomForest_4_[('n_estimators', 100), ('criterion', 'entropy'), ('max_depth', 13), ('min_samples_leaf', 1)]",
 "RandomForest_5_[('n_estimators', 150), ('criterion', 'gini'), ('max_depth', 5), ('min_samples_leaf', 9)]",
 "RandomForest_6_[('n_estimators', 650), ('criterion', 'entropy'), ('max_depth', 5), ('min_samples_leaf', 11)]",
 "RandomForest_7_[('n_estimators', 750), ('criterion', 'gini'), ('max_depth', 2), ('min_samples_leaf', 1)]",
 "RandomForest_8_[('n_estimators', 800), ('criterion', 'gini'), ('max_depth', 2), ('min_samples_leaf', 3)]",
 "RandomForest_9_[('n_estimators', 1250), ('criterion', 'gini'), ('max_depth', 1), ('min_samples_leaf', 5)]",
 

In [13]:
y_test = Test.TARGET
X_test = Test.drop('TARGET', axis=1)
print X_test.shape, ' ----- ', y_test.shape

(15204, 371)  -----  (15204L,)


In [ ]:
test_size

In [41]:
def countmean(preds, ensemble):
    mpreds = []
    for i in range(len(preds[0])):
        sum_p = 0.0
        mean = 0.0
        for row in preds:
            sum_p += row[i]
        mean = sum_p/len(ensemble)
        mpreds.append(mean)
        
    return mpreds
########################
def chekCorr(ensemble, m):
    state = False
    for subm in ensemble:
        if([subm, m] in couplesHighCorr):
            state = False
            break
        else:
            state = True  
    return state
########################
def findBestModel(ensemble, preds): #preds containes the predictions of the ensemble
    difference = list(set(header)-set(ensemble))
    temp1 = []
    temp2 = []
    scores = []
    temp1  = ensemble
    for m in difference:
        temp1.append(m)
        temp2.append(m)
        preds.append(dataFrame[m].values.tolist())
        mean_preds = countmean(preds, temp1)
        score = roc_auc_score(y_test, mean_preds)
        scores.append(score)
        temp1.pop()
    index = scores.index(max(scores))
    return temp2[index], max(scores)
###################################################################   

In [46]:
Learners_scores = pd.read_csv('sortedLearners.csv')
sorted_bag = [header[41], header[42], header[38], header[35], header[40], header[36], header[39], header[43+30], header[43+33],
              header[77+13], header[78], header[77+16], header[77+15], header[77+13], header[34], header[43+26], header[43+34],
              header[43+31], header[77+17]]
sorted_labels = ['RandomForest_42', 'RandomForest_43', 'RandomForest_39', 'RandomForest_41', 'RandomForest_36',
                 'RandomForest_37', 'RandomForest_40', 'GradientBoosting_30', 'GradientBoosting_33', 'AdaBoost_13',
                 'AdaBoost_1', 'AdaBoost_16', 'AdaBoost_15', 'RandomForest_35', 'GradientBoosting_26', 'GradientBoosting_34',
                 'GradientBoosting_31', 'AdaBoost_18', 'RandomForest_45', 'AdaBoost_21', 'AdaBoost_14', 'GradientBoosting_29',
                 'GradientBoosting_27', 'GradientBoosting_32', 'AdaBoost_17']
Learners_scores.head(25)

,Unnamed: 0,Learner,scoreAUC
0,41,RandomForest_42,0.838382
1,42,RandomForest_43,0.838064
2,38,RandomForest_39,0.837913
3,40,RandomForest_41,0.837631
4,35,RandomForest_36,0.837617
5,36,RandomForest_37,0.834804
6,39,RandomForest_40,0.833900
7,29,GradientBoosting_30,0.832060
8,32,GradientBoosting_33,0.830499
9,12,AdaBoost_13,0.829457


In [25]:
sorted_bag

["RandomForest_42_[('n_estimators', 1100), ('criterion', 'gini'), ('max_depth', 10), ('min_samples_leaf', 11)]",
 "RandomForest_43_[('n_estimators', 1450), ('criterion', 'entropy'), ('max_depth', 10), ('min_samples_leaf', 7)]",
 "RandomForest_39_[('n_estimators', 350), ('criterion', 'gini'), ('max_depth', 10), ('min_samples_leaf', 7)]",
 "RandomForest_36_[('n_estimators', 1400), ('criterion', 'entropy'), ('max_depth', 11), ('min_samples_leaf', 9)]",
 "RandomForest_41_[('n_estimators', 500), ('criterion', 'gini'), ('max_depth', 10), ('min_samples_leaf', 9)]",
 "RandomForest_37_[('n_estimators', 150), ('criterion', 'entropy'), ('max_depth', 13), ('min_samples_leaf', 9)]",
 "RandomForest_40_[('n_estimators', 50), ('criterion', 'entropy'), ('max_depth', 11), ('min_samples_leaf', 11)]",
 "GradientBoosting_30_[('n_estimators', 350), ('learning_rate', 0.03), ('max_depth', 2)]",
 "GradientBoosting_33_[('n_estimators', 300), ('learning_rate', 0.02), ('max_depth', 2)]",
 "AdaBoost_13_[('n_estima

In [48]:
%%time
#Add the sorted models to the ensmeble with their predictions proba
predicts = [] 
Bag = []
Labels_bag = []
scores_bag = []
for i, m in enumerate(sorted_bag):
    mean_pr = []
    Bag.append(m)
    Labels_bag.append(sorted_labels[i])
    predicts.append(dataFrame[m].values.tolist())
    mean_pr = countmean(predicts, Bag)
    score = roc_auc_score(y_test, mean_pr)
    print Labels_bag
    print ' >> ', score
    print '##################################"'
    scores_bag.append(score)
## Initialize the bag with top models FINISHED
print '*** FINISHED ***'  

['RandomForest_42']
 >>  0.838381806931
##################################"
['RandomForest_42', 'RandomForest_43']
 >>  0.838598547355
##################################"
['RandomForest_42', 'RandomForest_43', 'RandomForest_39']
 >>  0.83856797138
##################################"
['RandomForest_42', 'RandomForest_43', 'RandomForest_39', 'RandomForest_41']
 >>  0.838554949245
##################################"
['RandomForest_42', 'RandomForest_43', 'RandomForest_39', 'RandomForest_41', 'RandomForest_36']
 >>  0.838415299863
##################################"
['RandomForest_42', 'RandomForest_43', 'RandomForest_39', 'RandomForest_41', 'RandomForest_36', 'RandomForest_37']
 >>  0.838147981465
##################################"
['RandomForest_42', 'RandomForest_43', 'RandomForest_39', 'RandomForest_41', 'RandomForest_36', 'RandomForest_37', 'RandomForest_40']
 >>  0.83784659716
##################################"
['RandomForest_42', 'RandomForest_43', 'RandomForest_39', 'RandomForest

### Best 12 models > Score AUC == 0.838499995833

In [44]:
Bag

["RandomForest_42_[('n_estimators', 1100), ('criterion', 'gini'), ('max_depth', 10), ('min_samples_leaf', 11)]",
 "RandomForest_43_[('n_estimators', 1450), ('criterion', 'entropy'), ('max_depth', 10), ('min_samples_leaf', 7)]",
 "RandomForest_39_[('n_estimators', 350), ('criterion', 'gini'), ('max_depth', 10), ('min_samples_leaf', 7)]",
 "RandomForest_36_[('n_estimators', 1400), ('criterion', 'entropy'), ('max_depth', 11), ('min_samples_leaf', 9)]",
 "RandomForest_41_[('n_estimators', 500), ('criterion', 'gini'), ('max_depth', 10), ('min_samples_leaf', 9)]",
 "RandomForest_37_[('n_estimators', 150), ('criterion', 'entropy'), ('max_depth', 13), ('min_samples_leaf', 9)]",
 "RandomForest_40_[('n_estimators', 50), ('criterion', 'entropy'), ('max_depth', 11), ('min_samples_leaf', 11)]",
 "GradientBoosting_30_[('n_estimators', 350), ('learning_rate', 0.03), ('max_depth', 2)]",
 "GradientBoosting_33_[('n_estimators', 300), ('learning_rate', 0.02), ('max_depth', 2)]",
 "AdaBoost_13_[('n_estima

In [43]:
%%time
## Add the best model to maximize the score >>
ens = list(Bag)
pr = list(predicts)
for k in range(10):
    m, s = findBestModel(ens, pr)
    ens.append(m)
    pr.append(dataFrame[m].values.tolist())
    print m, ' --- ', s
print '*** FINISHED ***'  

DecisionTree_28_[('min_samples_split', 13), ('max_leaf_nodes', 7), ('criterion', 'entropy'), ('max_depth', 11), ('min_samples_leaf', 3)]  ---  0.837777840284
DecisionTree_4_[('min_samples_split', 5), ('max_leaf_nodes', 3), ('criterion', 'gini'), ('max_depth', 15), ('min_samples_leaf', 3)]  ---  0.818962521252
RandomForest_2_[('n_estimators', 250), ('criterion', 'gini'), ('max_depth', 10), ('min_samples_leaf', 9)]  ---  0.817818031803
RandomForest_2_[('n_estimators', 250), ('criterion', 'gini'), ('max_depth', 10), ('min_samples_leaf', 9)]  ---  0.817264643131
DecisionTree_4_[('min_samples_split', 5), ('max_leaf_nodes', 3), ('criterion', 'gini'), ('max_depth', 15), ('min_samples_leaf', 3)]  ---  0.816953674534
*** FINISHED ***
Wall time: 54min 37s
